In [39]:
# Uses Convolutional Neural Network to predict decibel level of sound

# @Note: that time is in datetime format which includes the date of compilation.

import numpy as np
import pandas as pd
from pandas import DataFrame as df

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
import keras.metrics

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import matplotlib.pyplot as plt
import os
import sys

# from utils import *

In [40]:
layercount = 3
layerunit = 64
batchsize = 24
epochs = 150
state = 15
testsize = 0.15

In [41]:
# Importing the dataset
dataset = pd.read_csv('data.csv')

# Shuffle the dataset
dataset = dataset.sample(frac=1)

# Drop ZIP code column
dataset = dataset.drop(['ZIP'], axis=1)

dataset.head()
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 157 to 115
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   INDEX                299 non-null    int64  
 1   START_TIME           299 non-null    object 
 2   LATITUDE             299 non-null    float64
 3   LONGITUDE            299 non-null    float64
 4   DISTANCE_TO_FREEWAY  299 non-null    float64
 5   Elevation            299 non-null    float64
 6   SOUND_SCORE          299 non-null    int64  
 7   NEAREST_ROAD         299 non-null    float64
 8   AVG_DB               299 non-null    int64  
dtypes: float64(5), int64(3), object(1)
memory usage: 23.4+ KB


In [42]:
# Convert #3 START_TIME (pandas Series Object) to datetime without date
dataset['START_TIME'] = pd.to_datetime(dataset['START_TIME']).apply(lambda x: x.value)

dataset.info()
dataset.head()
dataset.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 299 entries, 157 to 115
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   INDEX                299 non-null    int64  
 1   START_TIME           299 non-null    int64  
 2   LATITUDE             299 non-null    float64
 3   LONGITUDE            299 non-null    float64
 4   DISTANCE_TO_FREEWAY  299 non-null    float64
 5   Elevation            299 non-null    float64
 6   SOUND_SCORE          299 non-null    int64  
 7   NEAREST_ROAD         299 non-null    float64
 8   AVG_DB               299 non-null    int64  
dtypes: float64(5), int64(4)
memory usage: 23.4 KB


INDEX                  0
START_TIME             0
LATITUDE               0
LONGITUDE              0
DISTANCE_TO_FREEWAY    0
Elevation              0
SOUND_SCORE            0
NEAREST_ROAD           0
AVG_DB                 0
dtype: int64

In [43]:
# drops index
try:
    dataset = dataset.drop(['INDEX'], axis=1)
except:
    pass

print(dataset.head())

x = dataset.iloc[:,0:-1].values
y = dataset.iloc[:, -1].values
x, y

              START_TIME   LATITUDE   LONGITUDE  DISTANCE_TO_FREEWAY  \
157  1654659660000000000  32.696218 -117.101961             0.471405   
18   1654653780000000000  32.708869 -117.157771             0.808122   
159  1654658280000000000  32.700912 -117.123147             0.067344   
239  1654653000000000000  32.915384 -117.127303             0.606092   
214  1654654620000000000  32.853873 -117.193036             0.538748   

      Elevation  SOUND_SCORE  NEAREST_ROAD  AVG_DB  
157   22.027395           72      3.876486      65  
18     6.372311           65     47.843827      73  
159    6.950147           68     13.004439      69  
239  146.228851           71     24.192356      55  
214  113.212120           80     25.950255      51  


(array([[ 1.65465966e+18,  3.26962180e+01, -1.17101961e+02, ...,
          2.20273953e+01,  7.20000000e+01,  3.87648582e+00],
        [ 1.65465378e+18,  3.27088686e+01, -1.17157771e+02, ...,
          6.37231111e+00,  6.50000000e+01,  4.78438267e+01],
        [ 1.65465828e+18,  3.27009121e+01, -1.17123147e+02, ...,
          6.95014668e+00,  6.80000000e+01,  1.30044388e+01],
        ...,
        [ 1.65465492e+18,  3.30087744e+01, -1.17068605e+02, ...,
          2.08564484e+02,  7.30000000e+01,  1.56201676e+01],
        [ 1.65465858e+18,  3.27644414e+01, -1.17118654e+02, ...,
          1.22699356e+02,  7.40000000e+01,  5.62807682e+00],
        [ 1.65465270e+18,  3.27457720e+01, -1.17226022e+02, ...,
          3.77820778e+01,  7.00000000e+01,  1.44381950e+01]]),
 array([65, 73, 69, 55, 51, 65, 66, 53, 70, 71, 49, 64, 66, 66, 43, 52, 73,
        54, 65, 56, 64, 63, 57, 57, 55, 75, 72, 48, 65, 64, 56, 56, 62, 57,
        74, 54, 69, 67, 65, 61, 63, 65, 71, 60, 51, 64, 51, 62, 60, 56, 51,
 

In [44]:
import sklearn.preprocessing 
transformer = sklearn.preprocessing.StandardScaler()
# transformer = sklearn.preprocessing.MinMaxScaler()
# transformer = sklearn.preprocessing.MaxAbsScaler()
# transformer = sklearn.preprocessing.Normalizer()
# transformer = sklearn.preprocessing.RobustScaler()
# transformer = sklearn.preprocessing.QuantileTransformer()
x = transformer.fit_transform(x)
x

array([[-0.25451818, -1.08189953,  0.70033273, ..., -1.1306748 ,
         0.10613418, -1.92745206],
       [-0.64133828, -0.95234203, -0.39481467, ..., -1.38945085,
        -1.00456002,  2.5539361 ],
       [-0.34530248, -1.03382625,  0.28461282, ..., -1.37989932,
        -0.52854822, -0.99708194],
       ...,
       [-0.56634254,  2.11905749,  1.35486863, ...,  1.95275362,
         0.26480478, -0.73047277],
       [-0.32556677, -0.38320908,  0.37276652, ...,  0.53341671,
         0.42347538, -1.74892047],
       [-0.71238687, -0.5744064 , -1.73409388, ..., -0.8702524 ,
        -0.21120702, -0.85094579]])

In [45]:
from sklearn.model_selection import train_test_split
from keras import backend as K

def soft_acc(y_true, y_pred):
    return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = testsize, random_state=state)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((254, 7), (254,), (45, 7), (45,))

# K Nearest Neighbors

In [46]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
def print_grid_search_metrics(gs):
    print ("Best score: " + str(gs.best_score_))
    print ("Best parameters set:")
    best_parameters = gs.best_params_
    for param_name in sorted(best_parameters.keys()):
        print(param_name + ':' + str(best_parameters[param_name]))

In [47]:
classifier_KNN = KNeighborsClassifier()
# possible hyperparamter options for KNN
# choose k
parameters = {
    'n_neighbors': [1,3,5,7,9]
}

Grid_KNN = GridSearchCV(KNeighborsClassifier(), parameters, cv = 5)
Grid_KNN.fit(x_train, y_train)

c:\Python39\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9]})

In [48]:
from sklearn import neighbors
print_grid_search_metrics(Grid_KNN)

Best score: 0.0748235294117647
Best parameters set:
n_neighbors:1


# RSquared

In [49]:
def r2_score_from_scratch(ys_orig, ys_line):
    y_mean_line = [ys_orig.mean() for y in ys_orig]
    squared_error_regr = squared_error(ys_orig, ys_line)
    squared_error_y_mean = squared_error(ys_orig, y_mean_line)
    return 1 - (squared_error_regr / squared_error_y_mean)

def squared_error(ys_orig, ys_line):
    return sum((ys_line - ys_orig) * (ys_line - ys_orig))

for K in range(5):
    K = K + 1
    model = neighbors.KNeighborsRegressor(n_neighbors = K)
    model.fit(x_train, y_train) #fit the model
    pred=model.predict(x_test) #make prediction on test set
    r_squared = r2_score_from_scratch(y_test, pred)
    print('RSquared for K = ', K, ': ', r_squared)

RSquared for K =  1 :  0.10401627510171929
RSquared for K =  2 :  0.2628953930962067
RSquared for K =  3 :  0.30187688673054214
RSquared for K =  4 :  0.35625164063525383
RSquared for K =  5 :  0.37270507940674624


# MSE

In [50]:
from sklearn import neighbors
from sklearn.metrics import mean_squared_error
mse_val = [] #to store mse values for different k
for K in range(5):
    K = K + 1
    model = neighbors.KNeighborsRegressor(n_neighbors = K)

    model.fit(x_train, y_train) #fit the model
    pred=model.predict(x_test) #make prediction on test set
    error = mean_squared_error(y_test,pred) #calculate mse
    mse_val.append(error) #store mse values
    print('MSE value for k= ' , K , 'is:', error)

MSE value for k=  1 is: 33.71111111111111
MSE value for k=  2 is: 27.733333333333334
MSE value for k=  3 is: 26.266666666666666
MSE value for k=  4 is: 24.220833333333335
MSE value for k=  5 is: 23.60177777777777


# MAE


In [51]:
import statistics
mae_val = [] # to srore mae values for different k
for K in range(5):
    K = K + 1
    model = neighbors.KNeighborsRegressor(n_neighbors = K)
    
    model.fit(x_train, y_train) # fit the model
    pred=model.predict(x_test) # make prediction on test set
    error = abs(y_test - pred)# calculate mae
    mae_val.append(error)# store mae value
    print('MAE value for k= ', K , 'is:', error)
    average_mae = statistics.mean(error)
    print("Averageof MAE: ", average_mae)

MAE value for k=  1 is: [ 8.  6.  3.  0.  1.  2.  8.  7.  1.  4.  6.  4.  4.  3.  7.  4.  3.  6.
  3.  2.  3.  9. 11.  0.  1.  2.  4.  1.  1.  7.  2.  9. 12. 12.  4.  5.
  9.  8.  3.  0. 12.  2.  3.  4.  7.]
Averageof MAE:  4.733333333333333
MAE value for k=  2 is: [ 4.   4.   2.5  0.5  2.   0.  11.   3.   1.5  5.5  5.5  3.5  3.   5.5
 11.   3.   5.   6.   0.5  0.   1.   6.   8.   2.5  2.5  6.   1.   3.5
  5.   4.5  0.5  7.   5.   8.   4.   4.   8.   3.5  3.  10.   7.   4.
  4.5  0.5 11. ]
Averageof MAE:  4.377777777777778
MAE value for k=  3 is: [ 4.33333333  4.66666667  7.          0.66666667  2.          0.66666667
  7.33333333  6.          2.          4.          6.66666667  2.33333333
  5.33333333  6.33333333 11.33333333  3.33333333  3.          3.33333333
  1.33333333  2.          1.          6.          7.33333333  2.
  3.          5.33333333  0.33333333  2.33333333  5.66666667  5.33333333
  0.33333333  6.33333333  2.          7.66666667  4.66666667  4.66666667
  8.          0. 